In [123]:
import re
import logging
import pandas

In [6]:
temporary_file = 'stuff.dat'

In [71]:
instructions = 'twiss, range=#s/#e, sequence=swissfel,betx=betax0,bety=betay0,alfx=alphax0,alfy=alphay0,file="blabla.dat";'

# re.subn(r'^(twiss.*,file=")[a-z,A-Z,0-9,.]+(";)$', r'\1%s\2' % temporary_file, instructions,
#                       flags=re.MULTILINE)

pattern_twiss = re.compile(r'^(twiss.*,file=")[a-z,A-Z,0-9,.]+(";)$', flags=re.MULTILINE)
results = pattern_twiss.subn(r'\1%s\2' % temporary_file, instructions)
n_instructions = results[0]
match = results[1]

print(n_instructions)

twiss, range=#s/#e, sequence=swissfel,betx=betax0,bety=betay0,alfx=alphax0,alfy=alphay0,file="stuff.dat";


In [72]:
instructions = 'save,file="OptServValues.angle";\n'

# results = re.subn(r'^(save.*,file=")[a-z,A-Z,0-9,.]+(";)$', r'\1%s\2' % temporary_file, instructions,
#                       flags=re.MULTILINE)

pattern_save = re.compile(r'^(save.*,file=")[a-z,A-Z,0-9,.]+(";)$', flags=re.MULTILINE)
results = pattern_save.subn(r'\1%s\2' % temporary_file, instructions)
n_instructions = results[0]
match = results[1]

print(n_instructions)

save,file="stuff.dat";



In [73]:
instructions = 'write,table=r56data,file="OptServValues.R56";\n'

# results = re.subn(r'^(write.*,file=")[a-z,A-Z,0-9,.]+(";)$', r'\1%s\2' % temporary_file, instructions,
#                       flags=re.MULTILINE)

pattern_write = re.compile(r'^(write.*,file=")[a-z,A-Z,0-9,.]+(";)$', flags=re.MULTILINE)
results = pattern_write.subn(r'\1%s\2' % temporary_file, instructions)
n_instructions = results[0]
match = results[1]

print(n_instructions)

write,table=r56data,file="stuff.dat";



In [63]:
def resolve_type(formatting, value):
    if formatting.endswith('s'):
        return value.rstrip('"').lstrip('"')
    elif formatting == '%le':
        return float(value)
    else:
        logging.info('Cannot determine type of variable')
        return value
    

In [167]:
def read_twiss_write_data(filename):
    
    # Table will be
    # ^@ (global) variables - [name] [format] [value]
    # ^* name of column
    # ^$ format/type
    # values
    
    # Open data file from twiss and write
    with open(filename) as f:
        lines = f.readlines()

    global_variables = dict()
    pattern_global_variable = re.compile(r'^@\s+(\S+)\s+(%\S+)\s+(.+)$')

    pattern_column_names = re.compile(r'^\*\s+((\S+)\s+)*$')
    pattern_column_format = re.compile(r'^\$\s+((\S+)\s+)*$')
    pattern_data = re.compile(r'^\s*((\S+)\s*)*$')

    data_list = []

    for line in lines:
        # Check if lines holds global variables
        result = pattern_global_variable.match(line)
        if result:
            global_variables[result.group(1)] = resolve_type(result.group(2), result.group(3))
            continue

        # ---- DATA SECTION OF FILE

        # Check if line holds column names
        result = pattern_column_names.match(line)
        if result:
            columns = result.group(0).lstrip('*').split()
            continue;

        # Check if line holds column formats
        result = pattern_column_format.match(line)
        if result:
            column_formats = result.group(0).lstrip('$').split()
            continue;

        # Check if line holds data
        result = pattern_data.match(line)
        if result:
            data = line.split()
            # Resolve data type
            for i in range(len(data)):
                data[i] = resolve_type(column_formats[i], data[i])
            data_list.append(data)

    return global_variables, pandas.DataFrame(data_list, columns=columns)

In [176]:
table = read_twiss_write_data('tests/output.twiss')[1]

In [202]:
table.NAME
# table['NAME']
# table.columns
table.loc[table['NAME'] == 'DRIFT_1']
table.loc[0]

NAME    SINLH01$START
S                   0
BETX               25
ALFX                0
BETY               25
ALFY                0
X                   0
PX                  0
Y                   0
PY                  0
DX                  0
DPX                 0
DY                  0
DPY                 0
MUX                 0
MUY                 0
K1L                 0
Name: 1, dtype: object

In [85]:
lines

['    @ NAME             %05s "TWISS"\n',
 '@ TYPE             %05s "TWISS"\n',
 '@ SEQUENCE         %08s "SWISSFEL"\n',
 '@ PARTICLE         %08s "ELECTRON"\n',
 '@ MASS             %le      0.000510998928\n',
 '@ CHARGE           %le                  -1\n',
 '@ ENERGY           %le                 150\n',
 '@ PC               %le                 150\n',
 '@ GAMMA            %le         293542.6902\n',
 '@ KBUNCH           %le                   1\n',
 '@ BCURRENT         %le                   0\n',
 '@ SIGE             %le              0.0001\n',
 '@ SIGT             %le               0.001\n',
 '@ NPART            %le                   0\n',
 '@ EX               %le                   1\n',
 '@ EY               %le                   1\n',
 '@ ET               %le                   1\n',
 '@ LENGTH           %le          106.916288\n',
 '@ ALFA             %le                   0\n',
 '@ ORBIT5           %le                  -0\n',
 '@ GAMMATR          %le                   0\n',
 '@ Q

In [169]:
# Open data file from save
def read_save_data(filename):
    with open(filename) as f:
        lines = f.readlines()

    variables = dict()
    pattern_variable = re.compile(r'^(\S+)\s+:?=\s+(.+);$')

    for line in lines:
        result = pattern_variable.match(line)
        if result:
            variables[result.group(1)] = float(result.group(2))
    return variables

In [175]:
read_save_data('tests/output.save')

{'s10bd01.mqua010.k1': -0.6003796836,
 's10cb01.mqua230.k1': -1.493795661,
 's10cb01.mqua430.k1': 1.493419317,
 's10cb02.mqua230.k1': -1.4904,
 's10cb02.mqua430.k1': 1.489935,
 's10di01.mqua030.k1': 0.0,
 'sinbc01.mqua020.k1': 0.1711907615,
 'sinbc01.mqua050.k1': -0.09165118921,
 'sinbc01.mqua070.k1': -1.640221895,
 'sinbc01.mqua090.k1': 1.736390725,
 'sinbc01.mqua110.k1': 0.2048682463,
 'sinbc02.mqua110.k1': 0.0,
 'sinbc02.mqua120.k1': 0.0,
 'sinbc02.mqua340.k1': 0.0,
 'sinbc02.mqua350.k1': 0.0,
 'sindi01.mqua020.k1': -0.5326559539,
 'sindi01.mqua030.k1': 0.0,
 'sindi01.mqua070.k1': 0.2574976476,
 'sindi02.mqua020.k1': -2.747298848,
 'sindi02.mqua030.k1': 2.889927242,
 'sindi02.mqua050.k1': 0.5691890601,
 'sindi02.mqua060.k1': 0.7546769777,
 'sindi02.mqua070.k1': 0.0,
 'sindi02.mqua090.k1': -3.469474595,
 'sinlh01.mqua020.k1': 2.229435705,
 'sinlh01.mqua030.k1': 0.0,
 'sinlh01.mqua040.k1': -4.32859227,
 'sinlh01.mqua050.k1': 0.429362884,
 'sinlh01.mqua070.k1': 1.321585329,
 'sinlh02.m